In [ ]:
import pandas as pd

df_lyrics = pd.read_csv('./data/lyrics.csv')
#df_lyrics = pd.read_csv('./data/country_lyrics.csv')
df_lyrics.shape

In [ ]:
df_lyrics.head(5)

In [ ]:
df_country_lyrics = df_lyrics.where(df_lyrics.genre == 'Country').dropna().reset_index()
df_country_lyrics.shape

In [ ]:
df_country_lyrics.head(5)

In [ ]:
df_alan_jackson_lyrics = df_lyrics.where(df_lyrics.artist == 'alan-jackson').dropna().reset_index()
df_alan_jackson_lyrics.shape

In [ ]:
df_alan_jackson_lyrics.head(5)

In [ ]:
df_alan_jackson_lyrics.lyrics[0]

In [ ]:
!pip install -qU pip
!pip install -qU torch==1.4.0
!pip install -qU transformers==2.8.0

In [ ]:
""" 
Conditional text generation with the auto-regressive models of the library
(GPT/GPT-2/CTRL/Transformer-XL/XLNet)
"""

import logging

import numpy as np
import torch

from transformers import (
    CTRLLMHeadModel,
    CTRLTokenizer,
    GPT2LMHeadModel,
    GPT2Tokenizer,
    OpenAIGPTLMHeadModel,
    OpenAIGPTTokenizer,
    TransfoXLLMHeadModel,
    TransfoXLTokenizer,
    XLMTokenizer,
    XLMWithLMHeadModel,
    XLNetLMHeadModel,
    XLNetTokenizer,
)

logging.basicConfig(
    format="%(asctime)s - %(levelname)s - %(name)s -   %(message)s", datefmt="%m/%d/%Y %H:%M:%S", level=logging.INFO,
)
logger = logging.getLogger(__name__)

MAX_LENGTH = int(10000)  # Hardcoded max length to avoid infinite loop

MODEL_CLASSES = {
    "gpt2": (GPT2LMHeadModel, GPT2Tokenizer),
    "ctrl": (CTRLLMHeadModel, CTRLTokenizer),
    "openai-gpt": (OpenAIGPTLMHeadModel, OpenAIGPTTokenizer),
    "xlnet": (XLNetLMHeadModel, XLNetTokenizer),
    "transfo-xl": (TransfoXLLMHeadModel, TransfoXLTokenizer),
    "xlm": (XLMWithLMHeadModel, XLMTokenizer),
}

# Padding text to help Transformer-XL and XLNet with short prompts as proposed by Aman Rusia
# in https://github.com/rusiaaman/XLNet-gen#methodology
# and https://medium.com/@amanrusia/xlnet-speaks-comparison-to-gpt-2-ea1a4e9ba39e
PADDING_TEXT = """In 1991, the remains of Russian Tsar Nicholas II and his family
(except for Alexei and Maria) are discovered.
The voice of Nicholas's young son, Tsarevich Alexei Nikolaevich, narrates the
remainder of the story. 1883 Western Siberia,
a young Grigori Rasputin is asked by his father and a group of men to perform magic.
Rasputin has a vision and denounces one of the men as a horse thief. Although his
father initially slaps him for making such an accusation, Rasputin watches as the
man is chased outside and beaten. Twenty years later, Rasputin sees a vision of
the Virgin Mary, prompting him to become a priest. Rasputin quickly becomes famous,
with people, even a bishop, begging for his blessing. <eod> </s> <eos>"""


def set_seed(seed, n_gpu):
    np.random.seed(seed)
    torch.manual_seed(seed)
    if n_gpu > 0:
        torch.cuda.manual_seed_all(seed)

#
# Functions to prepare models' input
#


def prepare_ctrl_input(temperature, _, tokenizer, prompt_text):
    if temperature > 0.7:
        logger.info("CTRL typically works better with lower temperatures (and lower top_k).")

    encoded_prompt = tokenizer.encode(prompt_text, add_special_tokens=False)
    if not any(encoded_prompt[0] == x for x in tokenizer.control_codes.values()):
        logger.info("WARNING! You are not starting your generation from a control code so you won't get good results")
    return prompt_text


def prepare_xlm_input(xlm_language, model, tokenizer, prompt_text):
    # kwargs = {"language": None, "mask_token_id": None}

    # Set the language
    use_lang_emb = hasattr(model.config, "use_lang_emb") and model.config.use_lang_emb
    if hasattr(model.config, "lang2id") and use_lang_emb:
        available_languages = model.config.lang2id.keys()
        if xlm_language in available_languages:
            language = xlm_language
        else:
            language = None
            while language not in available_languages:
                language = input("Using XLM. Select language in " + str(list(available_languages)) + " >>> ")

        model.config.lang_id = model.config.lang2id[language]
        # kwargs["language"] = tokenizer.lang2id[language]

    # TODO fix mask_token_id setup when configurations will be synchronized between models and tokenizers
    # XLM masked-language modeling (MLM) models need masked token
    # is_xlm_mlm = "mlm" in args.model_name_or_path
    # if is_xlm_mlm:
    #     kwargs["mask_token_id"] = tokenizer.mask_token_id

    return prompt_text


def prepare_xlnet_input(padding_text, _, tokenizer, prompt_text):
     prompt_text = (padding_text if padding_text else PADDING_TEXT) + prompt_text
     return prompt_text


def prepare_transfoxl_input(padding_text, _, tokenizer, prompt_text):
    prompt_text = (padding_text if padding_text else PADDING_TEXT) + prompt_text
    return prompt_text


PREPROCESSING_FUNCTIONS = {
    "ctrl": 'blah',
    "xlm": 'blah',
    "xlnet": 'blah',
    "transfo-xl": 'blah',
}


def adjust_length_to_model(length, max_sequence_length):
    if length < 0 and max_sequence_length > 0:
        length = max_sequence_length
    elif 0 < max_sequence_length < length:
        length = max_sequence_length  # No generation bigger than model size
    elif length < 0:
        length = MAX_LENGTH  # avoid infinite loop
    return length


def generate(model_type, lyric):
    print(model_type)

    model_name_or_path = ''
    if model_type == 'gpt2':
        model_name_or_path='gpt2'

    if model_type == 'xlnet':
        model_name_or_path='xlnet-base-cased'

    if model_type == 'ctrl':
        model_name_or_path='ctrl'

    if model_type == 'xlm':
        model_name_or_path='xlm-mlm-en-2048'

    if model_type == 'transfo-xl':
        model_name_or_path='transfo-xl-wt103'

    print(model_name_or_path)

    length=20
    stop_token=None # Token at which text generation is stopped

    temperature=1.0 # Temperature of 1.0 has no effect, lower tend toward greedy sampling
    repetition_penalty=1.0 # Primarily useful for CTRL model; in that case, use 1.2
    k=0
    p=0.9
    padding_text='' # Padding text for Transfo-XL and XLNet.
    xlm_language='' # Optional language when used with the XLM model.
    seed=42 # Random seed for initialization
    no_cuda=True # Avoid using CUDA when available
    num_return_sequences=1 # The number of samples to generate
    device = torch.device("cuda" if torch.cuda.is_available() and not no_cuda else "cpu")
    n_gpu = 0 if no_cuda else torch.cuda.device_count()

    set_seed(seed, n_gpu)

    # Initialize the model and tokenizer
    try:
        model_type = model_type.lower()
        model_class, tokenizer_class = MODEL_CLASSES[model_type]
    except KeyError:
        raise KeyError("the model {} you specified is not supported. You are welcome to add it and open a PR :)")

    tokenizer = tokenizer_class.from_pretrained(model_name_or_path)
    model = model_class.from_pretrained(model_name_or_path)
    model.to(device)

    length = adjust_length_to_model(length, max_sequence_length=model.config.max_position_embeddings)

    prompt_text = lyric if lyric else input("Lyric? >>> ")

    # Different models need different input formatting and/or extra arguments
    requires_preprocessing = model_type in PREPROCESSING_FUNCTIONS.keys()

    preprocessed_prompt_text = None

    if requires_preprocessing:
        if model_type == "ctrl": 
            preprocessed_prompt_text = prepare_ctrl_input(temperature, model, tokenizer, prompt_text)
        if model_type == "xlm": 
            preprocessed_prompt_text = prepare_xlm_input(xlm_language, model, tokenizer, prompt_text)
        if model_type == "xlnet": 
            preprocessed_prompt_text = prepare_xlnet_input(padding_text, model, tokenizer, prompt_text)
        if model_type == "transfo-xl": 
            preprocessed_prompt_text = prepare_transfoxl_input(padding_text, model, tokenizer, prompt_text)

        encoded_prompt = tokenizer.encode(
            preprocessed_prompt_text, add_special_tokens=False, return_tensors="pt", add_space_before_punct_symbol=True
        )
    else:
        encoded_prompt = tokenizer.encode(prompt_text, add_special_tokens=False, return_tensors="pt")
    encoded_prompt = encoded_prompt.to(device)

    output_sequences = model.generate(
        input_ids=encoded_prompt,
        max_length=length + len(encoded_prompt[0]),
        temperature=temperature,
        top_k=k,
        top_p=p,
        repetition_penalty=repetition_penalty,
        do_sample=True,
        num_return_sequences=num_return_sequences,
    )

    # Remove the batch dimension when returning multiple sequences
    if len(output_sequences.shape) > 2:
        output_sequences.squeeze_()

    generated_sequences = []

    for generated_sequence_idx, generated_sequence in enumerate(output_sequences):
        print("=== GENERATED SEQUENCE {} ===".format(generated_sequence_idx + 1))
        generated_sequence = generated_sequence.tolist()

        # Decode text
        text = tokenizer.decode(generated_sequence, clean_up_tokenization_spaces=True)

        # Remove all text after the stop token
        text = text[: text.find(stop_token) if stop_token else None]

        # Add the prompt at the beginning of the sequence. Remove the excess text that was used for pre-processing
        total_sequence = (
            prompt_text + '\n**' + model_type + '**' + text[len(tokenizer.decode(encoded_prompt[0], clean_up_tokenization_spaces=True)) :] + '\n**' + model_type + '**\n'
        )

        generated_sequences.append(total_sequence)
        print(total_sequence)
        
    return generated_sequences


In [ ]:
generated_lyrics = generate(model_types=['gpt2'],
                            lyric="""I remember how you looked that day\nand thought we would be together\nuntil we were both old & gray\nbut, after 7 years of happiness & 8 month's of hell""")


In [ ]:
for generated_lyric in generated_lyrics:
    split_generated_lyrics = generated_lyric[0].split('\n')
    for split_generated_lyric in split_generated_lyrics:
        print(split_generated_lyric)

In [ ]:
generated_lyrics = generate(model_types=['xlnet'],
                            lyric="""I remember how you looked that day\nand thought we would be together\nuntil we were both old & gray\nbut, after 7 years of happiness & 8 month's of hell""")


In [ ]:
for generated_lyric in generated_lyrics:
    split_generated_lyrics = generated_lyric[0].split('\n')
    for split_generated_lyric in split_generated_lyrics:
        print(split_generated_lyric)

In [ ]:
generated_lyrics = generate(model_types=['ctrl'], # 'transfo-xl'], 
                            lyric="""I remember how you looked that day\nand thought we would be together\nuntil we were both old & gray\nbut, after 7 years of happiness & 8 month's of hell""")


In [ ]:
for generated_lyric in generated_lyrics:
    split_generated_lyrics = generated_lyric[0].split('\n')
    for split_generated_lyric in split_generated_lyrics:
        print(split_generated_lyric)

In [ ]:
generated_lyrics = generate(model_types=['xlm'], # 'transfo-xl'], 
                            lyric="""I remember how you looked that day\nand thought we would be together\nuntil we were both old & gray\nbut, after 7 years of happiness & 8 month's of hell""")


In [ ]:
for generated_lyric in generated_lyrics:
    split_generated_lyrics = generated_lyric[0].split('\n')
    for split_generated_lyric in split_generated_lyrics:
        print(split_generated_lyric)

In [ ]:
generated_lyrics = generate(model_types=['transfo-xl'], 
                            lyric="""I remember how you looked that day\nand thought we would be together\nuntil we were both old & gray\nbut, after 7 years of happiness & 8 month's of hell""")


In [ ]:
for generated_lyric in generated_lyrics:
    split_generated_lyrics = generated_lyric[0].split('\n')
    for split_generated_lyric in split_generated_lyrics:
        print(split_generated_lyric)